In [1]:
import os
from bs4 import BeautifulSoup

In [2]:
slozka = "filmy"

In [3]:
def seznam_souboru(slozka):
    seznam = []
    for file in os.listdir(slozka):
        seznam.append(file)
    return(seznam)

In [4]:
def id_persony(url):
    if "cs/person/" in url:
        id_persony = url.split("/")[-2]
    else:
        id_persony = 0
    return(id_persony)

In [5]:
def persona(odkaz):
    id_persony = f"""{odkaz.text.strip()} ({odkaz["href"].split("/")[-2]})"""
    return(id_persony)

In [6]:
import pandas as pd

In [7]:
def scrape_pisne(slozka, soubor):
    
    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
            
    pisne = pd.DataFrame()
                
    film = f"""{soup.find("h1").text.strip()} ({soup.find('meta', {'property': 'og:url'})['content'].split("/")[-2]})"""
    
    credit = soup.find(class_="film-credits-songs")

    if credit == None:
        
        pass
    
    else:
        
        pisnicky = credit.find_all('div', {'class': 'credit-item'})

        for p in pisnicky:
            
            pisen = []

            nazev = p.find("h3").text.strip()
            
            spany = p.find_all("span")
            
            hudba_k_pisni = []
            text_pisne = []
            zpiva = []

            kontrola = 0

            for s in spany:

                odkazy = s.find_all("a")

                if "ba k písni" in s.text.lower():

                    kontrola = 1
                    
                    pass

                if "xt pís" in s.text.lower():

                    kontrola = 2
                    
                    pass

                if s.text.strip().lower() == "zpívá":

                    kontrola = 3
                    
                    pass

                if kontrola == 1:

                    for o in odkazy:

                        if persona(o) not in hudba_k_pisni:

                            hudba_k_pisni.append(persona(o))
                            
                    pass

                if kontrola == 2:  

                    for o in odkazy:

                        if persona(o) not in text_pisne:

                            text_pisne.append(persona(o))
                            
                    pass

                if kontrola == 3:  

                    for o in odkazy:

                        if persona(o) not in zpiva:

                            zpiva.append(persona(o))
                            
                    pass
                
            if hudba_k_pisni == []:

                hudba_k_pisni = ""

            if text_pisne == []:

                text_pisne = ""

            if zpiva == []:

                zpiva = ""
                
            pisen = {"Film": film, "Píseň": nazev, "Hudba k písni": ", ".join(hudba_k_pisni), "Text písně": ", ".join(text_pisne), "Zpívá": ", ".join(zpiva)}
            
            pisen = pd.DataFrame(pisen, index=[0])
        
            pisne = pd.concat([pisne, pisen]).reset_index(drop = True)
                                
    return(pisne)

In [8]:
test = scrape_pisne(slozka, "396600-kdyby-tisic-klarinetu.html")
test

,Film,Píseň,Hudba k písni,Text písně,Zpívá
0,Kdyby tisíc klarinetů (396600),Kam utíkáš starý šneku,,,"Ivana Fišerová (31405), Rudolf Cortés (15020),..."
1,Kdyby tisíc klarinetů (396600),Babetta,Jiří Šlitr (2822),Jiří Suchý (2270),"Jiří Šlitr (2822), Jiří Suchý (2270)"
2,Kdyby tisíc klarinetů (396600),Tereza,Jiří Šlitr (2822),Jiří Suchý (2270),Waldemar Matuška (1787)
3,Kdyby tisíc klarinetů (396600),Píseň o rose,Jiří Šlitr (2822),Jiří Suchý (2270),Jiří Suchý (2270)
4,Kdyby tisíc klarinetů (396600),Spím,Jiří Šlitr (2822),Jiří Suchý (2270),"Karel Gott (3861), Jiří Suchý (2270), Jiří Šli..."
5,Kdyby tisíc klarinetů (396600),V opeře,Jiří Šlitr (2822),Jiří Suchý (2270),Hana Hegerová (13526)
6,Kdyby tisíc klarinetů (396600),Kde je ten míč,Jiří Šlitr (2822),Jiří Suchý (2270),"Jiří Suchý (2270), Jiří Šlitr (2822)"
7,Kdyby tisíc klarinetů (396600),Hallo Satchmo,Jiří Šlitr (2822),Jiří Suchý (2270),Jiří Jelínek (27512)
8,Kdyby tisíc klarinetů (396600),Tak abyste to věděla,Jiří Šlitr (2822),Jiří Suchý (2270),"Hana Hegerová (13526), Waldemar Matuška (1787)..."
9,Kdyby tisíc klarinetů (396600),Choď po špičkách,Jiří Šlitr (2822),Jiří Suchý (2270),Hana Hegerová (13526)


In [9]:
test = scrape_pisne(slozka, "103354-obcansky-prukaz.html")
print(test)
for x in test:
    print(len(x))

                        Film                           Píseň   
0   Občanský průkaz (103354)                Most of the Time  \
1   Občanský průkaz (103354)                  Obut a oblečen   
2   Občanský průkaz (103354)                        Paranoid   
3   Občanský průkaz (103354)                  Ballroom Blitz   
4   Občanský průkaz (103354)                Dobré ráno blues   
5   Občanský průkaz (103354)                         Nemakej   
6   Občanský průkaz (103354)                      Píseň kozi   
7   Občanský průkaz (103354)                    Eliášův oheň   
8   Občanský průkaz (103354)                           Lásko   
9   Občanský průkaz (103354)                Sun Is So Bright   
10  Občanský průkaz (103354)            Krajina posedlá tmou   
11  Občanský průkaz (103354)                        Joplinka   
12  Občanský průkaz (103354)                  Kovbojův nářek   
13  Občanský průkaz (103354)  Kdyby tady byla taková panenka   

                                       

In [10]:
test = scrape_pisne(slozka, "118284-dvojka.html")
test

,Film,Píseň,Hudba k písni,Text písně,Zpívá
0,Dvojka (118284),Funny,"a. m. almela (66179), Magdaléna Šalamounová (2...",,
1,Dvojka (118284),Tribute to Underworld,"DJ Smut (66180), Mawd (66181)",,
2,Dvojka (118284),Neon,Jan P. Muchow (2302),,
3,Dvojka (118284),Not As It Seems,Filip Míšek (9111),,
4,Dvojka (118284),No Name č. 1,Mawd (66181),,
5,Dvojka (118284),No Name č. 2,Mawd (66181),,
6,Dvojka (118284),Speedy Monkey,Mawd (66181),,
7,Dvojka (118284),A Sunday Afternoon on the Island of la Grande ...,"a. m. almela (66179), Magdaléna Šalamounová (2...",,
8,Dvojka (118284),Malá mořská víla,Aneta Langerová (19748),,
9,Dvojka (118284),Nicotine Lovers,"Karzi (26509), Gregory Finn (14726)",,


In [11]:
test = scrape_pisne(slozka, "510-rebelove.html")
test

,Film,Píseň,Hudba k písni,Text písně,Zpívá
0,Rebelové (510),Měsíc,Karel Mareš (2820),Eva Jarošová (2826),"Yvonne Přenosilová (910), Olympic (911)"
1,Rebelové (510),Gina,Josef Zíma (2821),Josef Zíma (2821),Josef Zíma (2821)
2,Rebelové (510),Pátá/Down Town,Tony Hatch (2863),Jiří Štaidl (2827),"Zuzana Norisová (581), The Vertigos (2865)"
3,Rebelové (510),Jezdím bez nehod,Jaroslav Mangl (2817),Zdeněk Borovec (2828),"Milan Chladil (2843), Risten kvartet (2844)"
4,Rebelové (510),"Jó, třešně zrály/Jailer, Give Me Water",,Ivo Fischer (2835),Waldemar Matuška (1787)
5,Rebelové (510),Letíme na Měsíc,Karel Duba (2816),Karel Duba (2816),"František Živný (2837), Libuše Příhodová (2838..."
6,Rebelové (510),"Sugar Town/Š,Š,Š",Lee Hazlewood (2861),Jan Fiala (2867),"Zuzana Norisová (581), The Vertigos (2865)"
7,Rebelové (510),Včera neděle byla,Jiří Šlitr (2822),Jiří Suchý (2270),Eduard Parma ml. (34460)
8,Rebelové (510),Oliver Twist,Karel Mareš (2820),Rostislav Černý (2866),"Anna Veselá (584), The Vertigos (2865)"
9,Rebelové (510),"Nej, nej, nej",,,"Pavlína Filipovská (22435), Josef Zíma (2821),..."


In [12]:
kolik = None

In [13]:
def frejm(slozka):
    pisnicky_pisnove = pd.DataFrame()
    total = len(seznam_souboru(slozka)[:kolik])
    count = 0
    for s in seznam_souboru(slozka)[:kolik]:
        count = count + 1
        print(f"{count} / {total} / {s}")
        line = (scrape_pisne(slozka, s))
        pisnicky_pisnove = pd.concat([pisnicky_pisnove, line])
    pisnicky_pisnove = pisnicky_pisnove.reset_index(drop = True)
    return(pisnicky_pisnove)

In [14]:
pisne_ceskych_filmu = frejm("filmy")

1 / 4562 / 10144-tabor-padlych-zen.html
2 / 4562 / 10173-eastern-krvavy-hugo.html
3 / 4562 / 102282-o-jecminkovi.html
4 / 4562 / 102314-zimni-vila.html
5 / 4562 / 10242-narkoblues.html
6 / 4562 / 103341-jan-hus-mse-za-tri-mrtve-muze.html
7 / 4562 / 103354-obcansky-prukaz.html
8 / 4562 / 103358-7-dni-hrichu.html
9 / 4562 / 103360-vojta-lavicka-nahoru-a-dolu.html
10 / 4562 / 103369-kuky-se-vraci.html
11 / 4562 / 103372-habermannuv-mlyn.html
12 / 4562 / 103374-dobre-placena-prochazka.html
13 / 4562 / 103402-nebe-peklo.html
14 / 4562 / 103495-lidice.html
15 / 4562 / 103499-medvedi-ostrovy.html
16 / 4562 / 103509-dont-stop.html
17 / 4562 / 104213-az-na-veky.html
18 / 4562 / 10574-elfilm.html
19 / 4562 / 106663-dum.html
20 / 4562 / 106666-soukromy-vesmir.html
21 / 4562 / 106667-mamas-papas.html
22 / 4562 / 10781-karu-suda-medvedi-srdce.html
23 / 4562 / 10960-unos-domu.html
24 / 4562 / 1118-nachove-plachty.html
25 / 4562 / 11221-mechanika.html
26 / 4562 / 112328-jmenem-krale.html
27 / 4562 / 

205 / 4562 / 180001-vsechno-je-sracka.html
206 / 4562 / 180025-laska-v-hrobe.html
207 / 4562 / 180317-rekvalifikace.html
208 / 4562 / 180386-muzi-v-nadeji.html
209 / 4562 / 18042-ctyri-lasky.html
210 / 4562 / 18045-fimfarum-jana-wericha.html
211 / 4562 / 180492-bastardi-2.html
212 / 4562 / 180545-svata-ctverice.html
213 / 4562 / 18056-franta-nebojsa.html
214 / 4562 / 18058-splneny-sen.html
215 / 4562 / 180585-okresni-prebor-posledni-zapas-pepika-hnatka.html
216 / 4562 / 180618-probudim-se-vcera.html
217 / 4562 / 181233-ve-stinu-bohyne-matky-zeme.html
218 / 4562 / 181912-zneuzivany.html
219 / 4562 / 181928-skolni-vylet.html
220 / 4562 / 181956-co-chytnes-v-zite.html
221 / 4562 / 181960-osina.html
222 / 4562 / 181961-cesta.html
223 / 4562 / 181962-stoh.html
224 / 4562 / 18203-z-mesta-cesta.html
225 / 4562 / 182231-libas-jako-dabel.html
226 / 4562 / 18262-pupendo.html
227 / 4562 / 18264-jedna-ruka-netleska.html
228 / 4562 / 18265-utek-do-budina.html
229 / 4562 / 182826-laska-je-laska.html

413 / 4562 / 395110-smich-a-plac.html
414 / 4562 / 395111-filmy-jana-krizeneckeho.html
415 / 4562 / 395111-vystavni-parkar-a-lepic-plakatu.html
416 / 4562 / 395112-to-nejlepsi-cislo.html
417 / 4562 / 395113-jarni-sen-stareho-mladence.html
418 / 4562 / 395114-hubicka.html
419 / 4562 / 395115-ponrepovo-kouzelnictvi.html
420 / 4562 / 395116-rudi-na-krtinach.html
421 / 4562 / 395117-rudi-na-zaletech.html
422 / 4562 / 395118-rudi-se-zeni.html
423 / 4562 / 395119-rudi-sportsman.html
424 / 4562 / 395120-sokove.html
425 / 4562 / 395121-ctyri-rocni-doby.html
426 / 4562 / 395122-dama-s-barzojem.html
427 / 4562 / 395123-falesny-hrac.html
428 / 4562 / 395124-pro-penize.html
429 / 4562 / 395125-zahadny-zlocin.html
430 / 4562 / 395126-zive-modely.html
431 / 4562 / 395127-americky-souboj.html
432 / 4562 / 395128-cholera-v-praze.html
433 / 4562 / 395129-estrella.html
434 / 4562 / 395130-faust.html
435 / 4562 / 395131-idyla-ze-stare-prahy.html
436 / 4562 / 395132-konec-milovani.html
437 / 4562 / 395133

615 / 4562 / 395312-mokre-dobrodruzstvi.html
616 / 4562 / 395313-on-a-ona.html
617 / 4562 / 395314-zkouska.html
618 / 4562 / 395315-bud-pripraven.html
619 / 4562 / 395316-carovne-oci.html
620 / 4562 / 395317-hrabenka-filangieri.html
621 / 4562 / 395318-kdyz-srdce-promluvi.html
622 / 4562 / 395319-madame-golvery.html
623 / 4562 / 395320-muz-bez-srdce.html
624 / 4562 / 395321-palicova-dcera.html
625 / 4562 / 395322-pepanek-nezdara.html
626 / 4562 / 395323-pro-a-proti.html
627 / 4562 / 395324-problematicky-gentleman.html
628 / 4562 / 395325-rasputin.html
629 / 4562 / 395326-tu-ten-kamen.html
630 / 4562 / 395327-unos-bankere-fuxe.html
631 / 4562 / 395328-za-oponou-smrti.html
632 / 4562 / 395329-zahadny-pripad-galginuv.html
633 / 4562 / 395330-zavet-podivinova.html
634 / 4562 / 395331-bily-raj.html
635 / 4562 / 395332-chytte-ho.html
636 / 4562 / 395333-dama-z-baru.html
637 / 4562 / 395334-devce-z-hor.html
638 / 4562 / 395335-dvoji-zivot.html
639 / 4562 / 395336-hrichy-v-manzelstvi.html
640 

813 / 4562 / 395511-pili-a-pokrokem-k-uspechu.html
814 / 4562 / 395512-pradlenka-jeho-jasnosti.html
815 / 4562 / 395513-tajemstvi-lekarovo.html
816 / 4562 / 395514-utrpeni-sede-sestry.html
817 / 4562 / 395515-v-kropenate-lhote-hori.html
818 / 4562 / 395516-vendelinuv-ocistec-a-raj.html
819 / 4562 / 395517-vse-pro-lasku.html
820 / 4562 / 395518-za-rodnou-hroudu.html
821 / 4562 / 395519-rada-lekare.html
822 / 4562 / 395520-tonka-sibenice.html
823 / 4562 / 395521-film-se-sasou-rasilovem-a-karlem-faltysem.html
824 / 4562 / 395522-kdyz-struny-lkaji-nemecka-verze.html
825 / 4562 / 395523-on-a-jeho-sestra-nemecka-verze.html
826 / 4562 / 395524-afera-plukovnika-redla-nemecka-verze.html
827 / 4562 / 395525-c-a-k-polni-marsalek-francouzska-verze.html
828 / 4562 / 395526-zena-ktera-se-smeje.html
829 / 4562 / 395527-svet-bez-hranic.html
830 / 4562 / 395528-afera-plukovnika-redla.html
831 / 4562 / 395529-pana-bumlicka-novy-domov.html
832 / 4562 / 395530-pensionat-petersilia.html
833 / 4562 / 395531

...

3740 / 4562 / 401684-slavnostni-vysveceni-mostu-cisare-frantiska-i.html
3741 / 4562 / 401685-cviceni-hasicu.html
3742 / 4562 / 401686-paty-vsesokolsky-slet-v-praze.html
3743 / 4562 / 401687-kocarova-doprava.html
3744 / 4562 / 401688-jizda-prahou-otevrenou-tramvaji.html
3745 / 4562 / 401689-jubilejni-hold-mestskych-sboru-prazskych.html
3746 / 4562 / 401690-jubilejni-vystava-obchodni-a-zivnostenske-komory-1908-v-praz.html
3747 / 4562 / 401691-kvetinove-kocary.html
3748 / 4562 / 401692-polni-mse-na-staromestskem-namesti.html
3749 / 4562 / 401693-pruvod-delegaci-v-tradicnich-krojich.html
3750 / 4562 / 401694-pruvod-sokolstva-v-prostejove.html
3751 / 4562 / 401695-prvni-den-jarnich-dostihu-prazskych.html
3752 / 4562 / 401696-prvni-sjezd-ceskych-streleckych-jednot-v-praze.html
3753 / 4562 / 401697-reklamni-pruvod-ceskych-podniku-a-spolecnosti.html
3754 / 4562 / 401698-slavnostni-a-vsedni-vyjevy-z-vystaviste.html
3755 / 4562 / 401699-slavnost-otevreni-noveho-cechova-mostu.html
3756 / 4562 / 4

3918 / 4562 / 401868-divotvorne-oko.html
3919 / 4562 / 401869-2-kaucuk.html
3920 / 4562 / 401869-2x-kaucuk.html
3921 / 4562 / 401870-emil-zatopek.html
3922 / 4562 / 401871-gregor-mendel.html
3923 / 4562 / 401872-hudebni-jaro.html
3924 / 4562 / 401873-chemie-v-kuchyni.html
3925 / 4562 / 401874-jak-se-dela-film.html
3926 / 4562 / 401875-junaci-v-prirode.html
3927 / 4562 / 401876-kapky-a-bubliny.html
3928 / 4562 / 401877-ladovi-furianti.html
3929 / 4562 / 401878-lide-a-parky.html
3930 / 4562 / 401879-miliony-na-dlazbach.html
3931 / 4562 / 401880-narodni-umelec-josef-lada.html
3932 / 4562 / 401881-nevime-dne.html
3933 / 4562 / 401882-obraz.html
3934 / 4562 / 401883-sklenena-suita.html
3935 / 4562 / 401884-to-jsou-bratri-v-triku.html
3936 / 4562 / 401885-cena-za-stesti.html
3937 / 4562 / 401886-letni-hokej.html
3938 / 4562 / 401887-podkarpatska-rus.html
3939 / 4562 / 401888-posadka-na-stite.html
3940 / 4562 / 401889-pred-startem-do-vesmiru.html
3941 / 4562 / 401890-ptak-ktery-kona-vojenskou

4107 / 4562 / 402063-sarkofag-pro-kralovnu.html
4108 / 4562 / 402064-meky.html
4109 / 4562 / 402065-prilis-osobni-znamost.html
4110 / 4562 / 402067-svina.html
4111 / 4562 / 402068-3bobule.html
4112 / 4562 / 402069-havel.html
4113 / 4562 / 402070-kralove-videa.html
4114 / 4562 / 402071-kiruna-prekrasny-novy-svet.html
4115 / 4562 / 402072-milenec-leta.html
4116 / 4562 / 402073-cesta-do-nemozna.html
4117 / 4562 / 402075-svet-podle-muchy.html
4118 / 4562 / 402077-poslouchej.html
4119 / 4562 / 402078-zenska-pomsta.html
4120 / 4562 / 402079-modelar.html
4121 / 4562 / 402080-mlsne-medvedi-pribehy-v-kinech.html
4122 / 4562 / 402081-v-siti-18.html
4123 / 4562 / 402082-alchymicka-pec.html
4124 / 4562 / 402083-jan-jedlicka-stopy-krajiny.html
4125 / 4562 / 402084-frem.html
4126 / 4562 / 402085-nebe.html
4127 / 4562 / 402086-nova-sichta.html
4128 / 4562 / 402087-zmatek.html
4129 / 4562 / 402088-provincni-mestecko-e.html
4130 / 4562 / 402089-anny.html
4131 / 4562 / 402090-vysehrad-seryjal.html
4132 

4304 / 4562 / 47585-cesky-fenomen-tanecni.html
4305 / 4562 / 48211-saxana-a-lexikon-kouzel.html
4306 / 4562 / 48348-pad.html
4307 / 4562 / 48378-oko-ve-zdi.html
4308 / 4562 / 48383-chytte-doktora.html
4309 / 4562 / 48718-marcela.html
4310 / 4562 / 48860-posledni-plavky.html
4311 / 4562 / 48950-vetrelci-v-coloradu.html
4312 / 4562 / 495-cabriolet.html
4313 / 4562 / 50479-jedne-noci-v-jednom-meste.html
4314 / 4562 / 50480-ulity.html
4315 / 4562 / 50482-jedne-noci-v-jednom-meste.html
4316 / 4562 / 50484-vetvicka-a-ploutvicka.html
4317 / 4562 / 510-rebelove.html
4318 / 4562 / 51759-tobruk.html
4319 / 4562 / 51760-medvidek.html
4320 / 4562 / 51777-bobule.html
4321 / 4562 / 52494-protektor.html
4322 / 4562 / 52699-tajnosti.html
4323 / 4562 / 52734-ivetka-a-hora.html
4324 / 4562 / 52854-edith-piaf.html
4325 / 4562 / 52996-praha-ocima.html
4326 / 4562 / 53009-karty-jsou-rozdany.html
4327 / 4562 / 53011-absolutni-laska.html
4328 / 4562 / 53015-obrazky-z-vyletu.html
4329 / 4562 / 53021-riziko.ht

4506 / 4562 / 8786-posledni-stisko-pana-zorze.html
4507 / 4562 / 8787-garden-party-s-vernisazi.html
4508 / 4562 / 88837-pripad-neverne-klary.html
4509 / 4562 / 8898-v-zahrade.html
4510 / 4562 / 8961-mladi-muzi-poznavaji-svet.html
4511 / 4562 / 8966-holcicky-na-zivot-a-na-smrt.html
4512 / 4562 / 8967-zahrada.html
4513 / 4562 / 89876-the-resolution.html
4514 / 4562 / 8999-mnaga-happy-end.html
4515 / 4562 / 9007-ceremoniar.html
4516 / 4562 / 9013-kolja.html
4517 / 4562 / 90196-cinka-panna.html
4518 / 4562 / 9024-drahy-mistre.html
4519 / 4562 / 9055-zapomenute-svetlo.html
4520 / 4562 / 90894-kratka-dlouha-cesta.html
4521 / 4562 / 9095-papirove-hlavy.html
4522 / 4562 / 9096-do-nebicka.html
4523 / 4562 / 9097-eine-kleine-jazzmusik.html
4524 / 4562 / 9098-o-trech-rytirich-krasne-pani-a-lnene-kytli.html
4525 / 4562 / 9099-kral-ubu.html
4526 / 4562 / 9101-co-delat-cesta-z-prahy-do-ceskeho-krumlova-aneb-jak-jsem-ses.html
4527 / 4562 / 9102-marian.html
4528 / 4562 / 9103-spiklenci-slasti.html
452

In [15]:
pisne_ceskych_filmu

,Film,Píseň,Hudba k písni,Text písně,Zpívá
0,Občanský průkaz (103354),Most of the Time,Bob Dylan (48949),,Bob Dylan (48949)
1,Občanský průkaz (103354),Obut a oblečen,Petr Ostrouchov (5833),,"Matouš Vrba (76615), Petr Ostrouchov (5833)"
2,Občanský průkaz (103354),Paranoid,"Tony Iommi (5649), Bill Ward (76634), Ozzy Osb...","Tony Iommi (5649), Bill Ward (76634), Ozzy Osb...",Black Sabbath (76649)
3,Občanský průkaz (103354),Ballroom Blitz,"Michael Chapman (76637), Nicholas Chinn (76638)","Michael Chapman (76637), Nicholas Chinn (76638)",Sweet (76650)
4,Občanský průkaz (103354),Dobré ráno blues,,Vladimír Ptaszek (76643),"Ľuboš Beňa (76632), Matěj Ptaszek (30591)"
...,...,...,...,...,...
10164,Baader Meinhof Komplex (99894),Dizzy Miss Lizzy,Larry E. Williams (51312),Larry E. Williams (51312),
10165,Baader Meinhof Komplex (99894),Child in Time,"Jon Lord (51313), Ian Gillan (51314), Roger Gl...","Jon Lord (51313), Ian Gillan (51314), Roger Gl...",skupina Deep Purple (51328)
10166,Baader Meinhof Komplex (99894),"Please, Please, Please",Michael Gerdes-Heitling (51317),"Michael Gerdes-Heitling (51317), Jens-Peter Fu...",
10167,Baader Meinhof Komplex (99894),Blowin' in the Wind,Bob Dylan (48949),Bob Dylan (48949),Bob Dylan (48949)


In [16]:
def do_seznamu(polozka):
    try:
        if "," in polozka:
            polozka = str(polozka).split(",")
            polozka = [p.strip() for p in polozka]
    except:
        pass
    return(polozka)

In [17]:
def znuluj(x):
    if x == "":
        y = None
    else:
        y = x
    return(y)

In [18]:
pisne_ceskych_filmu["Hudba k písni"] = pisne_ceskych_filmu["Hudba k písni"].apply(lambda x: do_seznamu(x))
pisne_ceskych_filmu["Text písně"] = pisne_ceskych_filmu["Text písně"].apply(lambda x: do_seznamu(x))
pisne_ceskych_filmu["Zpívá"] = pisne_ceskych_filmu["Zpívá"].apply(lambda x: do_seznamu(x))

In [19]:
pisne_ceskych_filmu["Hudba k písni"] = pisne_ceskych_filmu["Hudba k písni"].apply(lambda x: znuluj(x))
pisne_ceskych_filmu["Text písně"] = pisne_ceskych_filmu["Text písně"].apply(lambda x: znuluj(x))
pisne_ceskych_filmu["Zpívá"] = pisne_ceskych_filmu["Zpívá"].apply(lambda x: znuluj(x))

In [20]:
import os

In [21]:
pisne_ceskych_filmu.to_json(os.path.join("data", "pisne.json"), orient="records")